In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
data = pd.read_csv("../input/nasa-cmaps/CMaps/train_FD001.txt", sep=" ", header=None)

In [1]:
data.head()

In [1]:
COLS = ["ID", "Cycle", "OpSet1", "OpSet2", "OpSet3", "SensorMeasure1", "SensorMeasure2", "SensorMeasure3", "SensorMeasure4",
                "SensorMeasure5", "SensorMeasure6", "SensorMeasure7", "SensorMeasure8", "SensorMeasure9", "SensorMeasure10", "SensorMeasure11",
                "SensorMeasure12", "SensorMeasure13", "SensorMeasure14", "SensorMeasure15", "SensorMeasure16",
                "SensorMeasure17", "SensorMeasure18", "SensorMeasure19", "SensorMeasure20", "SensorMeasure21"]

In [1]:
# NaN olan sütunları çıkarıp başlıkları ekle
data = data[[f for f in range(0, 26)]]
data.columns = ["ID", "Cycle", "OpSet1", "OpSet2", "OpSet3", "SensorMeasure1", "SensorMeasure2", "SensorMeasure3", "SensorMeasure4",
                "SensorMeasure5", "SensorMeasure6", "SensorMeasure7", "SensorMeasure8", "SensorMeasure9", "SensorMeasure10", "SensorMeasure11",
                "SensorMeasure12", "SensorMeasure13", "SensorMeasure14", "SensorMeasure15", "SensorMeasure16",
                "SensorMeasure17", "SensorMeasure18", "SensorMeasure19", "SensorMeasure20", "SensorMeasure21"]

In [1]:
data

In [1]:
# Sadece 1 ID'li engine'in bilgileri
data[data["ID"]==1]

In [1]:
# Her engine'in kaç cycle dayandığının bilgisi
max_cycles_df = data.groupby(["ID"], sort=False)["Cycle"].max().reset_index().rename(columns={"Cycle" : "MaxCycleID"})
max_cycles_df.head()

In [1]:
# Remaining Useful Life (RUL) ile dataseti birleştir
FD001_df = pd.merge(data, max_cycles_df, how="inner", on="ID")
FD001_df["RUL"] = FD001_df["MaxCycleID"] - FD001_df["Cycle"]
FD001_df

In [1]:
# CSV'ye dönüştür
FD001_df.to_csv("FD001.csv", index=None)

# VERİ NORMALİZASYONU VE GÖRSELLEŞTİRME

In [1]:
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

In [1]:
df_mini = FD001_df.copy().drop(columns=["ID", "Cycle", "OpSet1", "OpSet2", "OpSet3", "MaxCycleID", "RUL"], axis=1)
scaler = MinMaxScaler(feature_range=(0, 1))
print(df_mini)
scaled_df = pd.DataFrame(scaler.fit_transform(df_mini.values))
scaled_df = FD001_df[["ID", "Cycle", "RUL"]].join(scaled_df)
scaled_df.columns = ["ID", "Cycle","RUL", "SensorMeasure1", "SensorMeasure2", "SensorMeasure3", "SensorMeasure4",
                "SensorMeasure5", "SensorMeasure6", "SensorMeasure7", "SensorMeasure8", "SensorMeasure9", "SensorMeasure10", "SensorMeasure11",
                "SensorMeasure12", "SensorMeasure13", "SensorMeasure14", "SensorMeasure15", "SensorMeasure16",
                "SensorMeasure17", "SensorMeasure18", "SensorMeasure19", "SensorMeasure20", "SensorMeasure21"]

In [1]:
df_mini.values

In [1]:
scaled_df

In [1]:
print("SCALED")
plt.figure()
scaled_df.plot.scatter(x="Cycle", y=["SensorMeasure4"])
plt.show()

print("ORIGINAL")
plt.figure()
data.plot.scatter(x="Cycle", y=["SensorMeasure4"])
plt.show()

In [1]:
plt.style.use("seaborn")
sensor_count = 21
engine_id = 2
for i in range(1, sensor_count+1):
    y_value = "SensorMeasure" + str(i)
    scaled_df[scaled_df["ID"]==engine_id].plot(x="Cycle", y=y_value)

**SensorMeasure 1, 5, 6, 10, 16, 18, 19 Gereksiz gibi görünüyor.**

In [1]:
scaled_clean_df = scaled_df.drop(columns=["SensorMeasure1", "SensorMeasure5", "SensorMeasure6", "SensorMeasure10",
                                         "SensorMeasure16", "SensorMeasure18", "SensorMeasure19"])
scaled_clean_df

In [1]:
cycle_counts = scaled_clean_df.groupby("ID")["Cycle"].max().reset_index()

In [1]:
cycle_counts.plot(x="ID", y=["Cycle"], kind="hist")
cycle_counts.plot.scatter(x="Cycle", y=["ID"])

In [1]:
standart_deviations = scaled_clean_df.groupby("ID").std().reset_index().drop(columns=["Cycle"])
standart_deviations

In [1]:
engine_id = 1
standart_deviations[standart_deviations["ID"]==engine_id].plot(y=["SensorMeasure2","SensorMeasure3","SensorMeasure4","SensorMeasure7",
                                                                 "SensorMeasure8","SensorMeasure9","SensorMeasure11","SensorMeasure12",
                                                                 "SensorMeasure13","SensorMeasure14","SensorMeasure15","SensorMeasure17",
                                                                 "SensorMeasure20","SensorMeasure21"], kind="bar", figsize=(10, 5))

In [1]:
# Genel görüntü
sensors = ["SensorMeasure2","SensorMeasure3","SensorMeasure4","SensorMeasure7",
           "SensorMeasure8","SensorMeasure9","SensorMeasure11","SensorMeasure12",
           "SensorMeasure13","SensorMeasure14","SensorMeasure15","SensorMeasure17",
           "SensorMeasure20","SensorMeasure21"]

engine_to_check = 1

scaled_clean_df[scaled_clean_df["ID"]==engine_to_check].plot(x="Cycle", y=sensors, figsize=(20, 10))

rul_fixed_df = scaled_clean_df.copy()
rul_fixed_df.loc[rul_fixed_df["RUL"] < 20, "RUL"] = 0
rul_fixed_df.loc[rul_fixed_df["RUL"] >= 20, "RUL"] = 1
rul_fixed_df
col = rul_fixed_df[rul_fixed_df["ID"]==engine_to_check].RUL.map({1:'b', 0:'r'})
rul_fixed_df[rul_fixed_df["ID"]==engine_to_check].plot.scatter(x=sensors[5], y=[sensors[9]], c=col)
rul_fixed_df[rul_fixed_df["ID"]==engine_to_check].plot.scatter(x=sensors[2], y=[sensors[6]], c=col)
rul_fixed_df[rul_fixed_df["ID"]==engine_to_check].plot.scatter(x=sensors[12], y=[sensors[13]], c=col)

In [1]:
import seaborn

In [1]:
correlation_df = scaled_clean_df.drop(columns=["ID", "Cycle", "RUL"])
corr_default = correlation_df.corr()
corr_default

In [1]:
plt.figure(figsize=(12, 10))
seaborn.heatmap(corr_default, annot=True, cmap="rocket")

In [1]:
correlation_df = scaled_df.drop(columns=["ID", "Cycle", "RUL"])
corr_default = correlation_df.corr()
corr_default

In [1]:
plt.figure(figsize=(12, 10))
seaborn.heatmap(corr_default, annot=True, cmap="rocket")

In [1]:
S1, S2 = "SensorMeasure11", "SensorMeasure12"
fig, axes = plt.subplots(2, 5, figsize=(15, 10))
fig.tight_layout()

for i in range(2):
    for j in range(5):
        df_used = scaled_clean_df[scaled_clean_df["ID"]==(5*i+j+1)]
        axes[i][j].plot(df_used["Cycle"], df_used[S1])
        axes[i][j].plot(df_used["Cycle"], df_used[S2])
        axes[i][j].title.set_text("Engine ID: " + str(5*i+j+1))

plt.show()

In [1]:
# Veri kümesi boyunca sensörlerin değerleri

fig, axes = plt.subplots(1, 2, figsize=(25, 10))
axes[0].title.set_text("Tüm veri kümesi")
seaborn.heatmap(scaled_clean_df.copy().drop(columns=["ID", "Cycle", "RUL"]), ax=axes[0])

engine_id = 1
axes[1].title.set_text("Motor ID: " + str(engine_id))
seaborn.heatmap(scaled_clean_df[scaled_clean_df["ID"]==engine_id].copy().drop(columns=["ID", "Cycle", "RUL"]), ax=axes[1])

In [1]:
# Veri kümesinden çıkardığım sensörleri geri ekleyince oluşan heatmap

fig, axes = plt.subplots(1, 2, figsize=(30, 10))
axes[0].title.set_text("Çıkarılan özelliklerle tüm veri kümesi")
seaborn.heatmap(scaled_df.copy().drop(columns=["ID", "Cycle", "RUL"]), ax=axes[0])

engine_id = 1
axes[1].title.set_text("Çıkarılan özelliklerle Motor ID: " + str(engine_id))
seaborn.heatmap(scaled_df[scaled_df["ID"]==engine_id].copy().drop(columns=["ID", "Cycle", "RUL"]), ax=axes[1])

# **VERILERI SINIFLANDIRMA**

In [1]:
class_df = scaled_clean_df.copy()
class_df['HS'] = [0 for x in range(len(class_df['RUL']))]
class_df.loc[class_df["RUL"] <= 10, "HS"] = 'Not Okay'
class_df.loc[class_df['RUL'] >= 120, 'HS'] = 'Okay'
class_df.loc[(class_df['RUL'] < 120) & (class_df['RUL'] > 10), 'HS'] = 'Degradation'
class_df

In [1]:
HS_numeric = class_df.HS.map({'Okay':2, 'Not Okay':0, 'Degradation':1})
col = class_df.HS.map({'Okay':'b', 'Not Okay':'r', 'Degradation':'y'})
class_df['HS Numeric'] = HS_numeric
plt.figure()
class_df.plot.scatter(x="Cycle", y=["HS Numeric"], color=col)

In [1]:
fig, axes = plt.subplots(4, 4, figsize=(15, 15))
engine = 1
sensor = "SensorMeasure9"
for i in range(4):
    for j in range(4):
        col = class_df[class_df['ID']==engine].HS.map({'Okay':'b', 'Not Okay':'r', 'Degradation':'y'})
        plt.figure()
        axes[i][j].scatter(class_df[class_df["ID"]==engine]["Cycle"], class_df[class_df["ID"]==engine][sensor], color=col)
        axes[i][j].title.set_text("Engine: " + str(engine))
        engine += 1


In [1]:
#dataset özeti
class_df.info

In [1]:
#kaç tane (target) okey not okey ve degradation var?
class_df['HS Numeric'].value_counts()

In [1]:
#feature vectörünü ve hedef değişkenini bildirelim
X = class_df.drop([ 'ID','Cycle','RUL','HS','HS Numeric'], axis=1)

y = class_df['HS Numeric']

In [1]:
#X ve y'yi eğitim ve test setlerine ayırın
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 42)

In [1]:
#X_train ve x_test in shapeini kontrol edelim
X_train.shape, X_test.shape

In [1]:
#X_traindeki dataların typelarına bakalım
X_train.dtypes

In [1]:
#all of the values are number
X_train.head()

In [1]:
# Kategori kodlayıcılarını import edelim (encoders)
import category_encoders as ce

In [1]:
# değişkenleri sıralı kodlamayla kodlayalım
#encoder = ce.OrdinalEncoder(cols=['HS'])

#X_train = encoder.fit_transform(X_train)

#X_test = encoder.transform(X_test)

In [1]:
X_train.head()

In [1]:
X_test.head()

In [1]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor

In [1]:
# kriter gini indeksi ile DecisionTreeClassifier modelini somutlaştıralım
clf_gini = DecisionTreeClassifier(criterion='gini', max_depth=10, random_state=0)


# fit the model
clf_gini.fit(X_train, y_train)

In [1]:
#Kriter gini indeksi ile Test seti sonuçlarını tahmin edelim
y_pred_gini = clf_gini.predict(X_test)

In [1]:
#Kriter gini indeksi ile accuracy score kontrol edin
from sklearn.metrics import accuracy_score

print('Model accuracy score with criterion gini index: {0:0.4f}'. format(accuracy_score(y_test, y_pred_gini)))

In [1]:
accuracy_score(y_test, y_pred_gini)

In [1]:
#Train ve test setlerin accuracy lerini karşılaştıralım

y_pred_train_gini = clf_gini.predict(X_train)

y_pred_train_gini

In [1]:
print('Training-set accuracy score: {0:0.4f}'. format(accuracy_score(y_train, y_pred_train_gini)))

In [1]:
#overfitting ve underfitting için bakıyoruz
#training ve test setin score larını print edelim

print('Training set score: {:.4f}'.format(clf_gini.score(X_train, y_train)))

print('Test set score: {:.4f}'.format(clf_gini.score(X_test, y_test)))

#ikiside aynı overfitting olabilir mi aşırı mı uyumlu sor?

In [1]:
#karar ağacını çizelim

#plt.figure(figsize=(12,8))

from sklearn import tree

#tree.plot_tree(clf_gini.fit(X_train, y_train))

In [1]:
# Karar ağacı Classifier (Criterion Entropy)
# DecisionTreeClassifier modelini ölçüt entropisi ile somutlaştır
clf_en = DecisionTreeClassifier(criterion='entropy', max_depth=10, random_state=0)


# fit the model
clf_en.fit(X_train, y_train)

In [1]:
#test set sonuçlarını criterion entropy ile tahmin edelim
y_pred_en = clf_en.predict(X_test)

In [1]:
#Criterion Entrop ile accuracy score u kontrol edelim
from sklearn.metrics import accuracy_score

print('Model accuracy score with criterion entropy: {0:0.4f}'. format(accuracy_score(y_test, y_pred_en)))

In [1]:
#Train ve Test setteki accuracy leri karşılaştıralım
y_pred_train_en = clf_en.predict(X_train)

y_pred_train_en

In [1]:
print('Training-set accuracy score: {0:0.4f}'. format(accuracy_score(y_train, y_pred_train_en)))

In [1]:
#overfitting ve underfitting kontrol edelim (train ve test sette)
print('Training set score: {:.4f}'.format(clf_en.score(X_train, y_train)))

print('Test set score: {:.4f}'.format(clf_en.score(X_test, y_test)))

In [1]:
#plt.figure(figsize=(12,8))

#from sklearn import tree

#tree.plot_tree(clf_en.fit(X_train, y_train))

In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.model_selection import GridSearchCV

# Choose the type of classifier. 
clf = RandomForestClassifier(random_state=42)

# Choose some parameter combinations to try
parameters = {'n_estimators': [10,20,50], 
              'max_features': ['log2', 'sqrt','auto'], 
              'criterion': ['entropy', 'gini'],
              'max_depth': [4, 5, 6, 10], 
              
             }

# Type of scoring used to compare parameter combinations
acc_scorer = make_scorer(accuracy_score)

grid_obj = GridSearchCV(estimator= clf, param_grid=parameters, cv= 5)
grid_obj.fit(X_train, y_train)
# Run the grid search
#grid_obj = GridSearchCV(clf, parameters, scoring=acc_scorer)
#grid_obj = grid_obj.fit(X_train, y_train)

# Set the clf to the best combination of parameters
clf = grid_obj.best_estimator_

# Fit the best algorithm to the data. 
clf.fit(X_train, y_train)

In [1]:
#predictions = clf.predict(X_test)
#print(accuracy_score(y_test, predictions))


In [1]:
grid_obj.best_params_

In [1]:
rfc1=RandomForestClassifier(random_state=42, max_features='log2', n_estimators= 20, max_depth=8, criterion='gini')

In [1]:
rfc1.fit(X_train, y_train)

In [1]:
pred=rfc1.predict(X_train)
print("Accuracy for Random Forest on data: ",accuracy_score(y_train,pred))